In [1]:
import pandas as pd
from classes.WindAPI import *
from sqlalchemy import *
from sqlalchemy.orm import Session
from create_ref_mariadb import mariadb_connection, windturbines
from create_ref_mongodb import mongodb_connection
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import numpy as np
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import pytz

In [3]:
def fetch_windturbines_data(engine):
    # Récupérer les identifiants et last scada update de turbines depuis la table windturbine dans MariaDB
    stmt = select(windturbines.c['windturbine_id','last_scada_update']).filter(windturbines.c.last_scada_update.isnot(None))
    turbines = []
    with Session(engine) as session:
        for row in session.execute(stmt):
            turbines.append(row)

    # Traiter les données SCADA pour chaque turbine
    return turbines

def fetch_scada_data(turbineId):
    pass

client_mongodb = mongodb_connection()
engine_mariadb = mariadb_connection()
turbines = fetch_windturbines_data(engine_mariadb)


tz = pytz.timezone('Europe/Paris')
current_datetime = datetime.now(tz)
lastday_datetime = current_datetime - timedelta(hours=24)


scada_col = client_mongodb.scada
for turbine in turbines:
    scada_data = scada_col.find({
        "windturbine_id": turbine.windturbine_id,
        "log_date": {'$gte': lastday_datetime}
    })
    df = pd.DataFrame(list(scada_data))
    print(df.head())

                        _id                          wind_turbine   
0  64818ee5b080353d9327b0c0  1ec6d7ce-93f7-6ef8-abda-53533bbf8f7e  \
1  64818ee5b080353d9327b0c1  1ec6d7ce-93f7-6ef8-abda-53533bbf8f7e   
2  64818ee5b080353d9327b0c2  1ec6d7ce-93f7-6ef8-abda-53533bbf8f7e   
3  64818ee5b080353d9327b0c3  1ec6d7ce-93f7-6ef8-abda-53533bbf8f7e   
4  64818ee5b080353d9327b0c4  1ec6d7ce-93f7-6ef8-abda-53533bbf8f7e   

             log_date  wind_speed_average  wind_direction  active_power   
0 2023-06-08 08:10:00                 3.0            65.6          -7.2  \
1 2023-06-08 08:00:00                 2.9            68.4          -7.2   
2 2023-06-08 07:50:00                 2.9            65.1          -8.2   
3 2023-06-08 07:40:00                 2.6            60.6          -8.3   
4 2023-06-08 07:30:00                 2.5            58.2          -7.7   

   rotor_speed  external_ambient_temperature  wind_speed_estimated   
0          1.8                          21.0                   3

KeyboardInterrupt: 